Quantified backlog state = Sum(Rule \* Weight \* 10)

This will give the team clear information about backlog state score in a number from 1-10 (1 - worst, 10 - best )
Weights need to sum up to 100%



In [ ]:
from jira import JIRA
import matplotlib.pyplot as plt
import pandas as pd
import re
from numpy import nan
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:


#to show all rules descriptions
pd.set_option('max_colwidth',200)

rules = pd.DataFrame()
rules['rule'] = ''
rules['weight'] = 0
rules['value'] = 0

rules['rule'] = ['In next 2 sprints there are items in Backlog state that SP sum is greater than estimated velocity, there are no items assigned to sprint that are not in Backlog state - Y/N',
                '#rule 1 Planned next 3 versions (all items estimated and in Backlog or started state (not in presprint) - Y/N',
                '% of Must, Urgent, Should items that are estimated regardless of status',
                '% of key milestone items estimated and in Backlog status']
rules['weight'] = [0.4, 0.25, 0.25, 0.1]

rules

Rules sum should be 1

In [ ]:
sum(rules.weight)

<div class="alert alert-block alert-success">
Set up nextSprint, currentVersion, nextVersion values.

They can be loaded automatically from Jira too.
</div>

In [ ]:
#nextSprints = ['IC - #4 20180219 - 1.13', 'IC - #5 20180305 - 1.14']

#velocity used to check if sprints are planned correctly
estimatedVelocity = 13

nextVersions = ['1.12', '1.13', '1.14', '1.15', '1.16']
milestones = ['Frimley MVP']

In [ ]:

jira_url = 'https://kainos-evolve.atlassian.net'
jira = JIRA(jira_url)

In [ ]:
#download next 2 sprints names
from jira.client import GreenHopper
options = {'server': jira_url}
gh = GreenHopper(options)
sprintsRaw = gh.sprints(285)

sprints = pd.DataFrame()
sprints['name'] = ''
sprints['state'] = ''

for sprint in sprintsRaw:
    sprints = sprints.append(
            {
                'name': sprint.name,
                'state': sprint.state
            }, ignore_index=True)
sprints = sprints.loc[(sprints['state'] == 'FUTURE')]
sprints.sort_values("name", inplace=True)

nextSprints = sprints.head(2)['name'].tolist()
nextSprints

In [ ]:
#rule 0
#In next 2 sprints there are items in Backlog state that SP sum is greater than estimated velocity, 
#there are no items assigned to sprint that are not in Backlog state - Y/N
#bugs don't have to be estimated

jql = 'sprint in ("{}") and type != Bug and status = Backlog and type not in subTaskIssueTypes()'.format('", "'.join(nextSprints))
jql

issuesRaw = jira.search_issues(jql)

issues = pd.DataFrame()
issues['key'] = ''
issues['sprint'] = ''
issues['SP'] = ''
issues['type'] = ''
issues['status'] = ''
issues['summary'] = ''

for issue in issuesRaw:
    for rawSprint in issue.fields.customfield_10007:
        #unfortunately sprint information is encoded and regex is needed
        matches = re.search('name=(.*?),', rawSprint)
        issues = issues.append(
            {
             'key': issue.key,
             'type': issue.fields.issuetype.name,
             'status': issue.fields.status.name,
             'SP' : issue.fields.customfield_10005,
             'summary': issue.fields.summary,
             'sprint' : matches.group(1)
            }, ignore_index=True)

issues.fillna(value=nan, inplace=True)
issues
#issues.fillna(0, inplace=True)


sprints = issues.groupby(['sprint']).agg({'SP':'sum'})
sprints = sprints.loc[(sprints['SP'] >= estimatedVelocity)]
sprints
#non zero SP sprints number should be equal to number of next sprints
rule_value = (len(sprints) == len(nextSprints))
rules.at[0, 'value'] = int(rule_value)

In [ ]:
#rule 1 Planned next 3 versions (all items estimated and in Backlog or started state (not in presprint) - Y/N
jql = 'fixVersion in ("{}") and type not in subTaskIssueTypes()'.format('", "'.join(nextVersions))
jql

issuesRaw = jira.search_issues(jql)

issues = pd.DataFrame()
issues['key'] = ''
issues['version'] = ''
issues['SP'] = ''
issues['type'] = ''
issues['status'] = ''
issues['summary'] = ''

#add issues to dataframe
for issue in issuesRaw:
    #issue may have many versions - in this approach, one version per issue is recommended
    for fixVersion in issue.fields.fixVersions:
        if(fixVersion.name in nextVersions):
            issues = issues.append(
                {'version': fixVersion.name, 
                 'key': issue.key,
                 'type': issue.fields.issuetype.name,
                 'status': issue.fields.status.name,
                 'SP': issue.fields.customfield_10005,
                 'summary': issue.fields.summary,
                 'team' : str(issue.fields.customfield_14200),
                }, ignore_index=True)
            
issues = issues.loc[~(issues['status'].isin(['Completed', 'Rejected']))]

#indicate not estimated issues, bugs don't have to be estimated
issues['isEstimated'] = False
issues.loc[(issues['type'].isin(['Bug', 'Epic'])), ['isEstimated']] = True
issues.loc[(issues['type'] != 'Bug') & ~(issues['SP'].isnull()), ['isEstimated']] = True

#indicate items in presprint
issues['inPresprint'] = False
issues.loc[(issues['status'].isin(['Awaiting Prioritisation', 'PO Refinement', 'UX Refinement', 'QA Refinement', 'Tech Refinement', 
                                  'Tech Refinement', 'Estimation'])), ['inPresprint']] = True
issues.sort_values("version", inplace=True)



issues = issues.loc[(issues['inPresprint'] == True) | (issues['isEstimated'] == False)]
issues

rule_value = len(issues) == 0
rules.at[1, 'value'] = int(rule_value)


In [ ]:
#rule 2
# % of Must, Urgent, Should items that are estimated regardless of status
# bugs don't have to be estimated
jql = 'project = VXT and priority in (Must,Urgent,Should) and type != Bug and type not in subTaskIssueTypes()'
jql

issuesRaw = jira.search_issues(jql)

issues = pd.DataFrame()
issues['key'] = ''
issues['version'] = ''
issues['SP'] = ''
issues['type'] = ''
issues['status'] = ''
issues['summary'] = ''
issues['priority'] = ''

#add issues to dataframe
for issue in issuesRaw:
    #issue may have many versions - in this approach, one version per issue is recommended
    for fixVersion in issue.fields.fixVersions:
        if(fixVersion.name in nextVersions):
            issues = issues.append(
                {'version': fixVersion.name, 
                 'key': issue.key,
                 'type': issue.fields.issuetype.name,
                 'status': issue.fields.status.name,
                 'SP': issue.fields.customfield_10005,
                 'summary': issue.fields.summary,
                 'priority' : str(issue.fields.priority.name),
                }, ignore_index=True)
issues

rule_value = round(len(issues.loc[~(issues['SP'].isnull())]) / len(issues), 2)
rules.at[2, 'value'] = rule_value

In [ ]:
# % of key milestone items estimated and in Backlog status
jql = 'project = VXT and type != Epic and status not in (Rejected, Completed) and type not in subTaskIssueTypes() and fixVersion in ("' + '", "'.join(milestones) + '")'
jql

issuesRaw = jira.search_issues(jql)

issues = pd.DataFrame()
issues['key'] = ''
issues['version'] = ''
issues['SP'] = ''
issues['type'] = ''
issues['status'] = ''
issues['summary'] = ''

#add issues to dataframe
for issue in issuesRaw:
    #issue may have many versions - in this approach, one version per issue is recommended
    for fixVersion in issue.fields.fixVersions:
        if(fixVersion.name in milestones):
            issues = issues.append(
                {'version': fixVersion.name, 
                 'key': issue.key,
                 'type': issue.fields.issuetype.name,
                 'status': issue.fields.status.name,
                 'SP': issue.fields.customfield_10005,
                 'summary': issue.fields.summary,
                }, ignore_index=True)
            
issues['isEstimated'] = False
issues.loc[(issues['type'] == 'Bug'), ['isEstimated']] = True
issues.loc[(issues['type'] != 'Bug') & ~(issues['SP'].isnull()), ['isEstimated']] = True
issues

rule_value = round(len(issues.loc[(issues['isEstimated'] == True)]) / len(issues), 2)
rule_value
rules.at[3, 'value'] = rule_value

In [ ]:
rules['score'] = rules.weight * rules.value * 10
rules

In [ ]:
score = round(sum(rules['score']), 2)


%store -r

if not 'scores' in globals():
    scores = pd.DataFrame()
    scores['date'] = ''
    scores['score'] = 0

scores = scores.append(
            {
             'date': pd.to_datetime('now'),
             'score': score,
            }, ignore_index=True)

%store scores

print("Backlog score {} / 10".format(score))

_ = plt.plot(scores['date'], scores['score'], "-r.")

_ = plt.xticks(rotation='vertical')
_ = plt.ylabel('Backlog score')
_ = plt.xlabel('Time')
axes = plt.gca()
axes.set_ylim([0,10])

